In [11]:
%%writefile main.py
import functions_framework
from flask import jsonify, request
import logging
import os
import sys
import pandas as pd
from google.cloud import storage
import io

# Add the current directory to the path to import any necessary modules
sys.path.append(os.path.dirname(os.path.realpath(__file__)))

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to read CSV from Google Cloud Storage
def read_csv_from_gcs(bucket_name, file_name):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    data = blob.download_as_bytes()
    df = pd.read_csv(io.BytesIO(data))
    return df

# Function to get customer data from the dataframe
def get_customer_data(df, customer_id):
    return df[df['CustomerID'].astype(str) == customer_id]

@functions_framework.http
def customer_lookup(request):
    """
    HTTP Cloud Function that handles customer lookup by customer ID from a CSV file in Google Cloud Storage.
    
    Args:
        request (flask.Request): The request object.
        
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`.
    """
    # Set CORS headers for the preflight request
    if request.method == 'OPTIONS':
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'POST',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }
        return ('', 204, headers)

    # Set CORS headers for the main request
    headers = {
        'Access-Control-Allow-Origin': '*'
    }

    try:
        # Parse the request JSON
        request_json = request.get_json(silent=True)
        if not request_json:
            logger.error("No JSON data in request")
            return jsonify({"status": "error", "message": "No JSON data in request"}), 400, headers
        
        # Get the customer_id from the request
        customer_id = request_json.get('customer_id')
        if not customer_id:
            logger.error("Missing customer_id in the request")
            return jsonify({"status": "error", "message": "Missing 'customer_id' in the request body"}), 400, headers
        
        # Fetch customer data from Google Cloud Storage
        bucket_name = 'csv_bank'  # Update with your GCS bucket name
        file_name = 'credit_data.csv'  # Update with your CSV file name
        
        df = read_csv_from_gcs(bucket_name, file_name)
        result_df = get_customer_data(df, str(customer_id))

        if result_df.empty:
            return jsonify({
                "status": "not_found",
                "message": f"No records found for customer_id {customer_id}."
            }), 404, headers

        records = result_df.to_dict(orient='records')

        return jsonify({
            "status": "success",
            "customer_id": customer_id,
            "records": records
        }), 200, headers

    except Exception as e:
        logger.error(f"Error processing request: {str(e)}")
        return jsonify({"status": "error", "message": f"Unexpected error occurred: {str(e)}"}), 500, headers


Overwriting main.py


In [13]:
# Write requirements.txt
with open("requirements.txt", "w") as f:
    f.write("pandas\ngoogle-cloud-storage\n")


In [14]:
!gcloud functions deploy customerLookup \
  --runtime python310 \
  --trigger-http \
  --entry-point customer_lookup \
  --allow-unauthenticated \
  --memory 512MB \
  --region us-central1

Preparing function...done.                                                     
Updating function (may take a while)...                                        
  . [Build]                                                                    
  . [Service]                                                                  
  . [ArtifactRegistry]                                                         
  . [Healthcheck]                                                              
  . [Triggercheck]                                                             
  Updating function (may take a while)...                                      





⠛ Updating function (may take a while)...                                      
  ⠛ [Build]                                                                    




⠹ Updating function (may take a while)...                                      
  ⠹ [Build] Build in progress... Logs are available at [https://console.cloud.g
  oogle.com/cloud-build/builds;